# 8. Закрепление знаний

In [1]:
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diab_df = diabetes.copy()
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


### Задание 8.1
Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [3]:
dupl_column = list(diab_df.columns)
#dupl_column.remove('Pregnancies')

mask = diab_df.duplicated(subset=dupl_column)
diab_duplicates = diab_df[mask]
print(f'Число найденных дубликатов: {diab_duplicates.shape[0]}')

diab_dedupped = diab_df.drop_duplicates(subset=diab_duplicates)
print(f'Результирующее число записей: {diab_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


### Задание 8.2
Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны.    
В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [4]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diab_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diab_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diab_dedupped[col].nunique() / diab_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
# Удаляем их с помощью метода drop(), передав результирующий список в его аргументы.
informations_diab_df = diab_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {informations_diab_df.shape[1]}')

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


### Задание 8.3
Попробуйте найти пропуски в данных с помощью метода isnull().

> **Спойлер**: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [5]:
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

def found_nan(data):
    return np.nan if data == 0 else data

for col in cols:
    informations_diab_df[col] = informations_diab_df[col].apply(found_nan)
    

informations_diab_df.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

### Задание 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [6]:

#задаем минимальный порог: вычисляем 70% от числа строк
thresh = informations_diab_df.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
informations_diab_df = informations_diab_df.dropna(thresh=thresh, axis=1)

print(informations_diab_df.shape[1])

8


### Задание 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [7]:
#отбрасываем строки с числом пропусков более 2 в строке
m = informations_diab_df.shape[1] #число признаков после удаления столбцов
daibet_df = informations_diab_df.dropna(thresh=m-2, axis=0)
daibet_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


### Задание 8.6

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [11]:
null_df = daibet_df.isnull().sum()
cols = null_df[null_df>0].index
for col in cols:
    daibet_df[col] = daibet_df[col].fillna(daibet_df[col].median())

print(daibet_df['SkinThickness'].mean().round(1))

29.1


### Задание 8.7

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [12]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице daibet_df и признаку SkinThickness, а также выведем размерности результатов:
outliers, cleaned = outliers_iqr(daibet_df, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


### Задание 8.8

Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [13]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице daibet_df и признаку SkinThickness, а также выведем размерности результатов:
outliers, cleaned = outliers_z_score(daibet_df, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757
